In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import kgbench as kg
import fire, sys

from kgbench import load, tic, toc, d


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors


#
from torch_geometric.utils import to_networkx
import networkx as nx

ModuleNotFoundError: No module named 'torch'

In [2]:
data = kg.load('micro', torch=True) 
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
data.triples

Number of entities: 5
Number of classes: 2
Types of relations: 2


tensor([[0, 0, 1],
        [1, 0, 2],
        [0, 0, 2],
        [2, 1, 3],
        [4, 1, 3],
        [4, 1, 0]], dtype=torch.int32)

In [3]:
def edge_index_oneadj(data):
    edge_index = torch.stack((data.triples[:, 0], data.triples[:, 2]),dim=0)
    return edge_index


In [10]:
# example sparse tensor with edge indices
edge_index = edge_index_oneadj(data)
print('edge index: ', edge_index)
#edge_index = torch.tensor([[0, 0, 1, 2], [1, 2, 0, 1]])

# create dictionary of node neighborhoods
neighborhoods = {}
for i in range(edge_index.max().item() + 1):
    neighborhoods[i] = []

# iterate over edge indices and populate neighborhoods dictionary
for j in range(edge_index.shape[1]):
    src, dst = edge_index[0, j].item(), edge_index[1, j].item()
    neighborhoods[src].append(dst)
    neighborhoods[dst].append(src)

print(neighborhoods)

edge index:  tensor([[0, 1, 0, 2, 4, 4],
        [1, 2, 2, 3, 3, 0]], dtype=torch.int32)
{0: [1, 2, 4], 1: [0, 2], 2: [1, 0, 3], 3: [2, 4], 4: [3, 0]}


In [14]:
import torch

# example sparse tensor with edge indices
#edge_index = torch.tensor([[0, 0, 1, 2], [1, 2, 0, 1]])

def find_n_hop_neighbors(edge_index, n):
    # create dictionary of node neighborhoods
    neighborhoods = {}
    for i in range(edge_index.max().item() + 1):
        neighborhoods[i] = set()

    # find 1-hop neighbors
    for j in range(edge_index.shape[1]):
        src, dst = edge_index[0, j].item(), edge_index[1, j].item()
        neighborhoods[src].add(dst)
        neighborhoods[dst].add(src)

    # find n-hop neighbors
    for k in range(2, n+1):
        for i in range(edge_index.max().item() + 1):
            neighbors = list(neighborhoods[i])
            for j in range(len(neighbors)):
                neighbor = neighbors[j]
                for l in neighborhoods[neighbor]:
                    neighborhoods[i].add(l)
                
    return neighborhoods

# test with n=2
neighborhoods = find_n_hop_neighbors(edge_index, n=2)
print(neighborhoods)


{0: {0, 1, 2, 3, 4}, 1: {0, 1, 2, 3, 4}, 2: {0, 1, 2, 3, 4}, 3: {0, 1, 2, 3, 4}, 4: {0, 1, 2, 3, 4}}
